In [52]:
import pandas as pd
import seaborn as sns
import pathlib as Path
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
import os
print(os.listdir("../input"))


['test', 'sample_submission', 'sample_submission.csv', 'test.csv', 'train.csv', 'train']


In [53]:
df_train = pd.read_csv('../input/train.csv')
df_train['pickup_datetime'] = pd.to_datetime(df_train['pickup_datetime'])
df_train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [54]:
df_train = df_train[df_train['passenger_count']>= 1]
df_train = df_train[df_train['trip_duration']<= 5000]

In [55]:
df_train['year'] = df_train['pickup_datetime'].dt.year
df_train['month'] = df_train['pickup_datetime'].dt.month
df_train['day'] = df_train['pickup_datetime'].dt.day
df_train['hour'] = df_train['pickup_datetime'].dt.hour
df_train['minute']= df_train['pickup_datetime'].dt.minute
df_train['second']= df_train['pickup_datetime'].dt.second

selected_columns = ['passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                   'dropoff_latitude','year','month','day','hour','minute',
                   'second']

In [56]:
X_train = df_train[selected_columns]
y_train = df_train['trip_duration']
X_train.shape, y_train.shape

((1454812, 11), (1454812,))

In [57]:
rf = RandomForestRegressor()
random_split = ShuffleSplit(n_splits=3, test_size=0.05, train_size=0.1, random_state=0)
looses = -cross_val_score(rf, X_train, y_train, cv=random_split, scoring='neg_mean_squared_log_error')
looses = [np.sqrt(l) for l in looses]
np.mean(looses)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.45144211822272035

In [58]:
rf.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [59]:
df_test = pd.read_csv('../input/test.csv')
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])

In [60]:
df_test['year'] = df_test['pickup_datetime'].dt.year
df_test['month'] = df_test['pickup_datetime'].dt.month
df_test['day'] = df_test['pickup_datetime'].dt.day
df_test['hour'] = df_test['pickup_datetime'].dt.hour
df_test['minute']= df_test['pickup_datetime'].dt.minute
df_test['second']= df_test['pickup_datetime'].dt.second

selected_columns = ['passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                   'dropoff_latitude','year','month','day','hour','minute',
                   'second']

In [61]:
X_test = df_test[selected_columns]


In [62]:
pred = rf.predict(X_test)
pred.mean()

844.310479353227

In [63]:
submission = pd.read_csv('../input/sample_submission.csv')
submission.head()


,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [64]:
submission['trip_duration'] = pred

In [65]:
submission.to_csv('submission.csv', index=False)